In [1]:
import pandas as pd
import os

In [3]:
file_location = '/home/jasonmellone/Documents/nyu/DSQF/analyst_forecast/'

In [17]:
# data_files = list(filter(lambda x: 'csv' in x, os.listdir(file_location)))

In [18]:
# data_files_full_path = list(map(lambda x: file_location + x,data_files))

In [32]:
# data = dict(zip(data_files,map(pd.read_csv,data_files_full_path)))

In [20]:
# data

In [31]:
# sp500 = pd.read_csv(file_location + 'AFE_data/data/clean_cons_70_17.csv')

<h2>Read in S&P constituent returns, and remove dates not existing in SPX index returns</h2>

In [30]:
spx_returns = pd.read_csv(file_location + 'SPX.csv',index_col=0)

In [8]:
sp500_constituent_returns = None
sp500_constituent_returns = pd.read_csv(file_location +'AFE_data/data/crsp_sp500_1970_2016.csv')
sp500_constituent_returns.date = pd.to_datetime(sp500_constituent_returns.date, format='%Y%m%d')
sp500_constituent_returns = sp500_constituent_returns[sp500_constituent_returns.date.isin(spx_returns.index)]

sp500_constituent_returns.to_pickle(file_location +'AFE_data/data/crsp_sp500_1970_2016_dates_cleaned.p')

<h2>Read in cleaned, unindexed IBES data</h2>

<b>FPEDATS</b>: Forecast Period End Date (needs to equal datadate; the end of the quarter)<br>
<b>ACTDATS</b>: Date forecast was entered into the database

In [36]:
ibes_estimates = pd.read_csv(file_location +  'AFE_data/ibes/clean_estimates.csv')
ibes_estimates.ACTDATS = pd.to_datetime(ibes_estimates.ACTDATS,format='%Y%m%d')
ibes_estimates.FPEDATS = pd.to_datetime(ibes_estimates.FPEDATS,format='%Y%m%d')
del ibes_estimates['quarternum']
del ibes_estimates['Unnamed: 0']
del ibes_estimates['MEASURE']

<b>PENDS</b>: Period end<br>
<b>ACTDATS</b>: Date forecast was entered into the database

In [48]:
ibes_actuals = pd.read_csv(file_location +  'AFE_data/ibes/clean_actuals.csv')
ibes_actuals.ACTDATS = pd.to_datetime(ibes_actuals.ACTDATS,format='%Y%m%d')
ibes_actuals.ANNDATS = pd.to_datetime(ibes_actuals.ANNDATS,format='%Y%m%d')
del ibes_actuals['MEASURE']
del ibes_actuals['Unnamed: 0']
del ibes_actuals['PDICITY']
del ibes_actuals['ANNTIMS']
del ibes_actuals['ACTTIMS']
del ibes_actuals['quarternum']
ibes_actuals = ibes_actuals[ibes_actuals.CURR_ACT=='USD']
del ibes_actuals['CURR_ACT']



In [ ]:
median_ibes_estimate_per_ticker = ibes_estimates.groupby(['CUSIP','ACTDATS']).agg({'VALUE':'median'}).reset_index()

In [55]:
mean_ibes_estimate_per_ticker = ibes_estimates.groupby(['CUSIP','ACTDATS']).agg({'VALUE':'mean'}).reset_index()

In [61]:
std_ibes_estimate_per_ticker = ibes_estimates.groupby(['CUSIP','ACTDATS']).agg({'VALUE':'std'}).reset_index().dropna()

In [63]:
actual_counts = ibes_actuals.groupby(['CUSIP','ACTDATS']).agg({'VALUE':'count'}).reset_index()

In [70]:
actual_counts[actual_counts.VALUE!=1]

,CUSIP,ACTDATS,VALUE
5,00080S10,2005-05-13,2
761,00790310,1987-04-13,2
2036,01941110,2000-06-06,2
2796,02742010,1990-11-15,2
3855,03748R10,1997-10-27,2
4326,03822W10,2007-01-25,2
4557,04040110,1996-05-28,3
4559,04040110,1997-03-27,3
4560,04040110,1997-09-04,2
4822,05361110,1991-02-16,2


In [68]:
ibes_actuals[(ibes_actuals.CUSIP=='92849710') & (ibes_actuals.ACTDATS==pd.to_datetime('2008-09-30'))].

,TICKER,CUSIP,OFTIC,CNAME,PENDS,ANNDATS,ACTDATS,VALUE
51013,VTSS,92849710,VTSS,VITESSE SEMICOND,20060331,2008-09-30,2008-09-30,-0.8
51014,VTSS,92849710,VTSS,VITESSE SEMICOND,20060630,2008-09-30,2008-09-30,-1.6
51016,VTSS,92849710,VTSS,VITESSE SEMICOND,20061231,2008-09-30,2008-09-30,-0.4
51017,VTSS,92849710,VTSS,VITESSE SEMICOND,20070331,2008-09-30,2008-09-30,0.0
51018,VTSS,92849710,VTSS,VITESSE SEMICOND,20070630,2008-09-30,2008-09-30,-0.2
51019,VTSS,92849710,VTSS,VITESSE SEMICOND,20070930,2008-09-30,2008-09-30,0.2
